In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
sns.set()

In [ ]:
### LSTM
# class Model:
#     def __init__(self, learning_rate, num_layers, size, size_layer, output_size, forget_bias = 0.1):
        
#         def lstm_cell(size_layer):
#             return tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)
#         rnn_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell(size_layer) for _ in range(num_layers)], state_is_tuple = False)
#         self.X = tf.placeholder(tf.float32, (None, None, size))
#         self.Y = tf.placeholder(tf.float32, (None, output_size))
#         drop = tf.contrib.rnn.DropoutWrapper(rnn_cells, output_keep_prob = forget_bias)
#         self.hidden_layer = tf.placeholder(tf.float32, (None, num_layers * 2 * size_layer))
#         self.outputs, self.last_state = tf.nn.dynamic_rnn(drop, self.X, initial_state = self.hidden_layer, dtype = tf.float32)
#         rnn_W = tf.Variable(tf.random_normal((size_layer, output_size)))
#         rnn_B = tf.Variable(tf.random_normal([output_size]))
#         self.logits = tf.matmul(self.outputs[-1], rnn_W) + rnn_B
#         self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
#         self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)


In [2]:
class Model:
    def __init__(self, learning_rate, num_layers, size, size_layer, output_size, forget_bias = 0.1):
        
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.BasicRNNCell(size_layer)
        backward_rnn_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell(size_layer) for _ in range(num_layers)], state_is_tuple = False)
        forward_rnn_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell(size_layer) for _ in range(num_layers)], state_is_tuple = False)
        self.X = tf.placeholder(tf.float32, (None, None, size))
        self.Y = tf.placeholder(tf.float32, (None, output_size))
        drop_backward = tf.contrib.rnn.DropoutWrapper(backward_rnn_cells, output_keep_prob = forget_bias)
        forward_backward = tf.contrib.rnn.DropoutWrapper(forward_rnn_cells, output_keep_prob = forget_bias)
        self.backward_hidden_layer = tf.placeholder(tf.float32, shape=(None, num_layers * size_layer))
        self.forward_hidden_layer = tf.placeholder(tf.float32, shape=(None, num_layers * size_layer))
        self.outputs, self.last_state = tf.nn.bidirectional_dynamic_rnn(forward_backward, drop_backward, 
                                                                        self.X, 
                                                                        initial_state_fw = self.forward_hidden_layer, 
                                                                        initial_state_bw = self.backward_hidden_layer,
                                                                        dtype = tf.float32)
        self.outputs = tf.concat(self.outputs, 2)
        rnn_W = tf.Variable(tf.random_normal((size_layer * 2, output_size)))
        rnn_B = tf.Variable(tf.random_normal([output_size]))
        self.logits = tf.matmul(self.outputs[-1], rnn_W) + rnn_B
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)

In [3]:
# df5 = pd.read_csv("Y5levelWT3layer17-13-9.csv",header=None)
dfmain = pd.read_csv("Data_merged.csv",index_col=0)
Close = dfmain["Close Price"]
df6 = pd.read_csv("Y5levelWT3layer18-14-10.csv",header=None)
df6[10] = Close

In [4]:
df = df6.copy()

In [5]:
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()
df.head()

0         1         2         3         4         5         6   \
0  0.504333  0.020622  0.091492  0.994347  0.062468  0.753422  0.052363   
1  0.501698  0.026658  0.115132  0.990695  0.081316  0.703286  0.069166   
2  0.485075  0.060177  0.152833  0.973803  0.133416  0.658505  0.111763   
3  0.464489  0.116995  0.182066  0.944323  0.187973  0.661275  0.156666   
4  0.442971  0.214841  0.237523  0.874115  0.255235  0.647284  0.220174   

         7         8         9        10  
0  0.081885  0.827858  0.050652  1284.91  
1  0.108327  0.810352  0.061496  1261.52  
2  0.168914  0.781748  0.100574  1262.90  
3  0.228894  0.752713  0.158890  1252.31  
4  0.314977  0.713180  0.250511  1273.70

In [6]:
minmax = MinMaxScaler().fit(df.iloc[:, 1:].astype('float32'))
df_log = minmax.transform(df.iloc[:, 1:].astype('float32'))
df_log = pd.DataFrame(df_log)
df_log.head()

num_layers = 1
size_layer = 128
timestamp = 5
epoch = 500
dropout_rate = 0.5
future_day = 50

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
tf.reset_default_graph()
modelnn = Model(0.01, num_layers, df_log.shape[1], size_layer, df_log.shape[1], dropout_rate)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(epoch):
    init_value_forward = np.zeros((1, num_layers * size_layer))
    init_value_backward = np.zeros((1, num_layers * size_layer))
    total_loss = 0
    for k in range(0, (df_log.shape[0] // timestamp) * timestamp, timestamp):
        batch_x = np.expand_dims(df_log.iloc[k: k + timestamp, :].values, axis = 0)
        batch_y = df_log.iloc[k + 1: k + timestamp + 1, :].values
        last_state, _, loss = sess.run([modelnn.last_state, 
                                        modelnn.optimizer, 
                                        modelnn.cost], feed_dict={modelnn.X: batch_x, 
                                                                  modelnn.Y: batch_y, 
                                                                  modelnn.backward_hidden_layer: init_value_backward,
                                                                 modelnn.forward_hidden_layer: init_value_forward})
        init_value_forward = last_state[0]
        init_value_backward = last_state[1]
        total_loss += loss
    total_loss /= (df_log.shape[0] // timestamp)
    if (i + 1) % 100 == 0:
        print('epoch:', i + 1, 'avg loss:', total_loss)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
epoch: 100 avg loss: 0.7569960208542376
epoch: 200 avg loss: 0.860832425449268
epoch: 3

In [9]:
output_predict = np.zeros((df_log.shape[0] + future_day, df_log.shape[1]))
output_predict[0, :] = df_log.iloc[0, :]
upper_b = (df_log.shape[0] // timestamp) * timestamp
init_value_forward = np.zeros((1, num_layers * size_layer))
init_value_backward = np.zeros((1, num_layers * size_layer))
for k in range(0, (df_log.shape[0] // timestamp) * timestamp, timestamp):
    out_logits, last_state = sess.run([modelnn.logits, modelnn.last_state], feed_dict = {modelnn.X:np.expand_dims(df_log.iloc[k: k + timestamp, :], axis = 0),
                                     modelnn.backward_hidden_layer: init_value_backward, modelnn.forward_hidden_layer: init_value_forward})
    init_value_forward = last_state[0]
    init_value_backward = last_state[1]
    output_predict[k + 1: k + timestamp + 1, :] = out_logits
    
out_logits, last_state = sess.run([modelnn.logits, modelnn.last_state], feed_dict = {modelnn.X:np.expand_dims(df_log.iloc[upper_b: , :], axis = 0),
                                     modelnn.backward_hidden_layer: init_value_backward, modelnn.forward_hidden_layer: init_value_forward})
init_value_forward = last_state[0]
init_value_backward = last_state[1]
output_predict[upper_b + 1: df_log.shape[0] + 1, :] = out_logits
df_log.loc[df_log.shape[0]] = out_logits[-1, :]
date_ori.append(date_ori[-1]+timedelta(days=9))



In [10]:
for i in range(future_day - 1):
    out_logits, last_state = sess.run([modelnn.logits, modelnn.last_state], feed_dict = {modelnn.X:np.expand_dims(df_log.iloc[-timestamp:, :], axis = 0),
                                     modelnn.backward_hidden_layer: init_value_backward, modelnn.forward_hidden_layer: init_value_forward})
    init_value_forward = last_state[0]
    init_value_backward = last_state[1]
    output_predict[df_log.shape[0], :] = out_logits[-1, :]
    df_log.loc[df_log.shape[0]] = out_logits[-1, :]
    date_ori.append(date_ori[-1]+timedelta(days=9))

In [11]:
df_log = minmax.inverse_transform(output_predict)
date_ori=pd.Series(date_ori).dt.strftime(date_format='%Y-%m-%d').tolist()